### MI Churn Assignment  
Submitted by Renita Kurian - PES1UG20CS331

In [1]:
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import seaborn as sns
from time import time
from datetime import datetime
from random import triangular

In [2]:
df= pd.read_csv('/content/dataset.csv')
#df.head(5)

#convert str to date
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
#df.count()

#df.isnull().sum()
df=df[df['CustomerID'].notna()]
#df.isnull().sum()

start_date=pd.Timestamp(2010,12,1)
end_date=pd.Timestamp(2011,8,31)
train_set=df[df['InvoiceDate']>=start_date]
train_set=train_set[train_set['InvoiceDate']<=end_date]
#train_set.head()

start_date=pd.Timestamp(2011,9,1)
end_date=pd.Timestamp(2011,12,31)
churn_set=df[df['InvoiceDate']>=start_date]
churn_set=churn_set[churn_set['InvoiceDate']<=end_date]
#churn_set.head()
train_unique=train_set['CustomerID'].unique()
churn_unique=churn_set['CustomerID'].unique()

ischurnlis=[]
for id in train_unique:
    if id in churn_unique:
        ischurnlis.append(id)

train_set=train_set.assign(isChurn=[1 if x in ischurnlis else 0 for x in train_set['CustomerID']])

#taking care of country
countries_dic={}
countries=train_set['Country'].unique()
for i,y in enumerate(countries):
    countries_dic[y]=i
countries_dic

train_set['Country']=[countries_dic[x] for x in train_set['Country']]
train_set['Country'].unique()
#we assign country codes instead of country names
train_set=train_set.drop(['Description'],axis=1)
train_set['StockCode']=train_set['StockCode'].str.replace('\D', '')
train_set=train_set[train_set['StockCode']!='']
train_set['StockCode']=train_set['StockCode'].astype(int)
train_set['CustomerID']=train_set['CustomerID'].astype(int)

# Dropping invoice no
train_set=train_set.drop(['InvoiceNo'],axis=1)

#converting date to features
train_set['month']=train_set['InvoiceDate'].dt.month.astype(int)
train_set['year']=train_set['InvoiceDate'].dt.year.astype(int)
train_set['day']=train_set['InvoiceDate'].dt.day.astype(int)
train_set['hour']=train_set['InvoiceDate'].dt.hour.astype(int)

train_set=train_set.drop(['InvoiceDate'],axis=1)

#normalising train set
from hmac import trans_36
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler1=StandardScaler()
scaler2=MinMaxScaler()

train_set['StockCode']=scaler2.fit_transform(train_set[['StockCode']])
train_set['CustomerID']=scaler2.fit_transform(train_set[['CustomerID']])
train_set['Country']=scaler2.fit_transform(train_set[['Country']])

train_set['Quantity']=scaler1.fit_transform(train_set[['Quantity']])
train_set['UnitPrice']=scaler1.fit_transform(train_set[['UnitPrice']])
train_set['month']=scaler1.fit_transform(train_set[['month']])
train_set['year']=scaler1.fit_transform(train_set[['year']])
train_set['day']=scaler1.fit_transform(train_set[['day']])
train_set['hour']=scaler1.fit_transform(train_set[['hour']])


#dropping useless cols
train_set=train_set.drop(['year'],axis=1)
y=train_set['isChurn']
x=train_set.drop(['isChurn'],axis=1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

X_train,X_val,y_train,y_val = sk.model_selection.train_test_split(X_train,y_train,test_size=0.2,random_state=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
#decision Tree
from sklearn.tree import DecisionTreeClassifier

model1=DecisionTreeClassifier()
model1.fit(X_train,y_train)

y_pred = model1.predict(X_test)
#evaluating decision tree
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[11657   358]
 [  398 29253]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     12015
           1       0.99      0.99      0.99     29651

    accuracy                           0.98     41666
   macro avg       0.98      0.98      0.98     41666
weighted avg       0.98      0.98      0.98     41666



In [6]:
#SVM
from sklearn.svm import LinearSVC

model2=LinearSVC(max_iter=5000,verbose=True)
model2.fit(X_train,y_train)

y_pred = model2.predict(X_test)

#evaluating SVM
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[LibLinear][[    0 12015]
 [    2 29649]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     12015
           1       0.71      1.00      0.83     29651

    accuracy                           0.71     41666
   macro avg       0.36      0.50      0.42     41666
weighted avg       0.51      0.71      0.59     41666



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [5]:
#ANN
from sklearn.neural_network import MLPClassifier

model3=MLPClassifier(max_iter=5000,verbose=True)
model3.fit(X_train,y_train)

y_pred = model3.predict(X_test)

#evaluating neural network
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Iteration 1, loss = 0.59713069
Iteration 2, loss = 0.59124795
Iteration 3, loss = 0.58924268
Iteration 4, loss = 0.58736420
Iteration 5, loss = 0.58534838
Iteration 6, loss = 0.58338757
Iteration 7, loss = 0.58195912
Iteration 8, loss = 0.58020079
Iteration 9, loss = 0.57925963
Iteration 10, loss = 0.57798848
Iteration 11, loss = 0.57691576
Iteration 12, loss = 0.57598360
Iteration 13, loss = 0.57519778
Iteration 14, loss = 0.57438377
Iteration 15, loss = 0.57360173
Iteration 16, loss = 0.57306145
Iteration 17, loss = 0.57255727
Iteration 18, loss = 0.57181841
Iteration 19, loss = 0.57140800
Iteration 20, loss = 0.57081912
Iteration 21, loss = 0.57035113
Iteration 22, loss = 0.56991542
Iteration 23, loss = 0.56958240
Iteration 24, loss = 0.56875310
Iteration 25, loss = 0.56839540
Iteration 26, loss = 0.56788380
Iteration 27, loss = 0.56732533
Iteration 28, loss = 0.56691367
Iteration 29, loss = 0.56671241
Iteration 30, loss = 0.56636632
Iteration 31, loss = 0.56578688
Iteration 32, los